In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon, MultiPolygon
from geoalchemy2 import Geometry, WKTElement
import matplotlib.pyplot as plt
from sqlalchemy import text

# Task 1

### Data Loading

In [2]:
# Reading csv
businesses = pd.read_csv("Businesses.csv")
income = pd.read_csv("Income.csv")
population = pd.read_csv("Population.csv")
stops = pd.read_csv("Stops.txt")

In [3]:
# Reading shapefiles 
catchments_primary = gpd.read_file("Catchments/catchments_primary.shp")
catchments_future = gpd.read_file("Catchments/catchments_future.shp")
catchments_secondary = gpd.read_file("Catchments/catchments_secondary.shp")
sa2_regions = gpd.read_file("SA2_2021_SHP/SA2_2021_AUST_GDA2020.shp")

In [4]:
# Summarise all dataframes using .info()
def df_summary(dataframes: dict):
    with open("df_summary.txt", "w") as f:
        for name,dataframe in dataframes.items():
            f.write(f"{name} summary:")
            dataframe.info(buf=f)
            f.write('-'*50+'\n') #separate output for clarity

In [5]:
dataframes = {"Catchments_secondary": catchments_secondary , "Catchments_future": catchments_future,
              "Catchments_primary": catchments_primary, "Businesses":businesses,
              "Income":income, "Stops":stops, "Population": population, "Sa2_regions":sa2_regions}

df_summary(dataframes)

In [6]:

def check_null_values(dataframes: dict):
    """check the different null variants in datasets
         show which col the null variant appears in and how many times"""
    null_variants = ['na', 'np'," ", "n/a", "N/A", "NULL", None]
    for name, dataframe in dataframes.items():
        null_details = {} 
        for col in dataframe.columns:
            col_data = dataframe[col]
            # create dict with null variant: number of occurences as key: value pair"
            null_counts =  {variant: (col_data == variant).sum() for variant in null_variants if variant in col_data.values}
            if any(null_counts.values()):
                 null_details[col] = null_counts
        if null_details:
            print(f"Null values in DataFrame {name}: ")
            for col, variants in null_details.items():
                print(f"Column: {col}")
                for variant, count in variants.items():
                    print(f" - '{variant}' appears {count} times")
            print("-"*50)

In [7]:
check_null_values(dataframes)

Null values in DataFrame Income: 
Column: earners
 - 'np' appears 7 times
Column: median_age
 - 'np' appears 7 times
Column: median_income
 - 'np' appears 7 times
Column: mean_income
 - 'np' appears 7 times
--------------------------------------------------
Null values in DataFrame Sa2_regions: 
Column: geometry
 - 'None' appears 19 times
--------------------------------------------------


### Data cleaning

In [8]:
# clean up np values in income df
income.drop(income[income['median_income']=='np'].index, inplace=True)

In [9]:
# drop rows with total people < 100 in population
population.drop(population[population['total_people'] <100].index, inplace=True)

In [10]:
# Code adapted from NB tutorial wk 8
#Renaming the latitude and longitude columns in stops df
stops.rename(columns={'stop_lat':'lat', 'stop_lon':'lng'}, inplace=True)

# Creating the geometry column (Code from NB tutorial Week 8)
stops['geom'] = gpd.points_from_xy(stops.lng, stops.lat)

# Dropping the old lat and lng columns
stops.drop(columns=['lat', 'lng', 'wheelchair_boarding', 'platform_code', 'location_type', 'stop_code'], inplace=True)

In [11]:
# Drop irrelevant columns from the catchments df 
dataframes = [catchments_future,catchments_primary,catchments_secondary]
columns_to_drop = ['KINDERGART','YEAR1','YEAR2','YEAR3',
                    'YEAR4','YEAR5','YEAR6','YEAR7',
                    'YEAR8','YEAR9','YEAR10','YEAR11',
                    'YEAR12', 'ADD_DATE']
for dataframe in dataframes: 
    if 'PRIORITY' in dataframe.columns:
         columns_to_drop.append('PRIORITY')
         dataframe.drop(columns_to_drop, axis=1, inplace=True)
    else: 
        dataframe.drop(columns_to_drop, axis=1, inplace=True)
        

In [12]:
# rename columns in the catchments dataframes
catchments_primary = catchments_primary.rename({'USE_ID':'useid', 'CATCH_TYPE': 'catchtype', 'USE_DESC':'use_desc', 'geometry':'geometry'},axis=1)

catchments_secondary = catchments_secondary.rename({'USE_ID':'useid', 'CATCH_TYPE': 'catchtype', 'USE_DESC':'use_desc', 'geometry':'geometry'},axis=1)
catchments_future = catchments_future.rename({'USE_ID':'useid', 'CATCH_TYPE': 'catchtype', 'USE_DESC':'use_desc', 'geometry':'geometry'},axis=1)

In [13]:
# Rename columns in the population df
population = population.rename({
    'sa2_code': 'sa2code', 'sa2_name': 'sa2name', '0-4_people': 'age_0_to_4', 
    '5-9_people': 'age_5_to_9', '10-14_people':'age_10_to_14', '15-19_people':'age_15_to_19' ,
    '20-24_people': 'age_20_to_24', '25-29_people': 'age_25_to_29', '30-34_people': 'age_30_to_34',
    '35-39_people': 'age_35_to_39', '40-44_people':'age_40_to_44', '45-49_people':'age_45_to_49',
    '50-54_people': 'age_50_to_54','55-59_people': 'age_55_to_59' , '60-64_people': 'age_60_to_64',
    '65-69_people': 'age_65_to_69', '70-74_people':'age_70_to_74', '75-79_people': 'age_75_to_79',
    '80-84_people': 'age_80_to_84', '85-and-over_people': 'age_85_and_over', 'total_people':' total_people'
        
    },axis=1)

In [14]:
# Filter the sa2_regions df for entries in Greater Sydney
sa2_regions.drop(sa2_regions[sa2_regions['GCC_NAME21']!='Greater Sydney'].index, inplace=True) # only 373 entries

# Check for null values
print(sa2_regions.isna().sum())

SA2_CODE21    0
SA2_NAME21    0
CHG_FLAG21    0
CHG_LBL21     0
SA3_CODE21    0
SA3_NAME21    0
SA4_CODE21    0
SA4_NAME21    0
GCC_CODE21    0
GCC_NAME21    0
STE_CODE21    0
STE_NAME21    0
AUS_CODE21    0
AUS_NAME21    0
AREASQKM21    0
LOCI_URI21    0
geometry      0
dtype: int64


In [15]:
# Drop irrelevant and repetitive columns from sa2_regions df
sa2_regions.drop(columns=['CHG_FLAG21','CHG_LBL21', 'LOCI_URI21', 'STE_NAME21', 'STE_CODE21', 'AUS_CODE21', 'AUS_NAME21', 'GCC_CODE21','GCC_NAME21'], inplace=True)

In [16]:
# rename columns
sa2_regions = sa2_regions.rename({'SA2_CODE21':'sa2code', 'SA2_NAME21':'sa2name', 'SA3_CODE21':'sa3code', 'SA3_NAME21':'sa3name', 
                                  'SA4_CODE21':'sa4code', 'SA4_NAME21':'sa4name', 'AREASQKM21':'areasqkm', 'geometry':'geometry'},axis=1)

### SRID Transformations

In [17]:
# Code from Tutorial NB week 8
srid = 4283
stops['geom'] = stops['geom'].apply(lambda x: WKTElement(x.wkt, srid=srid))

In [18]:
#Code from Tutorial NB week 8
def create_wkt_element(geom, srid):
    if geom.geom_type == 'Polygon':
        geom = MultiPolygon([geom])
    return WKTElement(geom.wkt, srid)

In [19]:
dataframes = [catchments_future,catchments_primary,catchments_secondary,sa2_regions]
srid = 4283

# Code modified from Tutorial NB week 8
for dataframe in dataframes:
    df_og = dataframe.copy()  # creating a copy of the original for later
    dataframe['geom'] = dataframe['geometry'].apply(lambda x: create_wkt_element(geom=x,srid=srid))  # applying the create_wkt_element\n",
    dataframe.drop(columns='geometry', inplace=True)  # deleting the geometry column"

In [20]:
from sqlalchemy import create_engine
import psycopg2
import psycopg2.extras
import json

credentials = "Credentials.json"

def pgconnect(credential_filepath, db_schema="public"):
    with open(credential_filepath) as f:
        db_conn_dict = json.load(f)
        host       = db_conn_dict['host']
        db_user    = db_conn_dict['user']
        db_pw      = db_conn_dict['password']
        default_db = db_conn_dict['user']
        port       = db_conn_dict['port']
        try:
            db = create_engine(f'postgresql+psycopg2://{db_user}:{db_pw}@{host}:{port}/{default_db}', echo=False)
            conn = db.connect()
            print('Connected successfully.')
        except Exception as e:
            print("Unable to connect to the database.")
            print(e)
            db, conn = None, None
        return db,conn

def query(conn, sqlcmd, args=None, df=True):
    result = pd.DataFrame() if df else None
    try:
        if df:
            result = pd.read_sql_query(sqlcmd, conn, params=args)
        else:
            result = conn.execute(text(sqlcmd), args).fetchall()
            result = result[0] if len(result) == 1 else result
    except Exception as e:
        print("Error encountered: ", e, sep='\n')
    return result

In [21]:
db, conn = pgconnect(credentials)

Connected successfully.


In [22]:
conn.execute(text("SET search_path TO public"))

In [23]:
query(conn, "select PostGIS_Version()")

,postgis_version
0,3.5 USE_GEOS=1 USE_PROJ=1 USE_STATS=1


### Ingesting into SQL

In [25]:
conn.rollback()
conn.execute(text("""
    DROP TABLE IF EXISTS income;
    CREATE TABLE income (
        sa2_code21 INTEGER UNIQUE,
        sa2_name VARCHAR(100),
        earners INTEGER, 
        median_age INTEGER,
        median_income INTEGER,
        mean_income INTEGER
    );"""))

conn.execute(text("""
    DROP TABLE IF EXISTS stops;
    CREATE TABLE stops (
        stopid INTEGER UNIQUE,  
        stop_code INTEGER, 
        stop_name VARCHAR(100),
        parent_station INTEGER,
        geom GEOMETRY(POINT,4283)
    );"""))

conn.execute(text("""
    DROP TABLE IF EXISTS businesses;
    CREATE TABLE businesses (
         industry_code CHAR(1),
         industry_name VARCHAR(100),
         sa2_code INTEGER,
         sa2_name VARCHAR(100),
         rng_0_to_50k_businesses INTEGER,
         rng_50k_to_200k_businesses INTEGER,
         rng_200k_to_2m_businesses INTEGER,
         rng_2m_to_5m_businesses INTEGER,
         rng_5m_to_10m_businesses INTEGER,
         rng_10m_or_more_businesses INTEGER,
         total_businesses INTEGER
    );"""))

conn.execute(text("""
    DROP TABLE IF EXISTS population;
    CREATE TABLE population (
        sa2_code INTEGER, 
        sa2_name VARCHAR(100), 
        age_0_to_4 INTEGER,
        age_5_to_9 INTEGER, 
        age_10_to_14 INTEGER,
        age_15_to_19 INTEGER,
        age_20_to_24 INTEGER, 
        age_25_to_29 INTEGER, 
        age_30_to_34 INTEGER,
        age_35_to_39 INTEGER, 
        age_40_to_44 INTEGER, 
        age_45_to_49 INTEGER, 
        age_50_to_54 INTEGER,
        age_55_to_59 INTEGER, 
        age_60_to_64 INTEGER, 
        age_65_to_69 INTEGER, 
        age_70_to_74 INTEGER,
        age_75_to_79 INTEGER, 
        age_80_to_84 INTEGER, 
        age_85_and_over INTEGER,
        total_people INTEGER
    );"""
))            

conn.execute(text("""
    DROP TABLE IF EXISTS sa2_regions CASCADE;
    CREATE TABLE sa2_regions (
         sa2code INTEGER,
         sa2name VARCHAR(100), 
         sa3code INTEGER,
         sa3name VARCHAR(100),
         sa4code INTEGER,
         sa4name VARCHAR(100),
         areasqkm FLOAT, 
         geom GEOMETRY(MULTIPOLYGON, 4283)
        );"""
))
conn.execute(text("""
    DROP TABLE IF EXISTS catchments_primary;
    CREATE TABLE catchments_primary (
         useid INTEGER,
         catchtype VARCHAR(100),
         usedesc VARCHAR(100),
         geom GEOMETRY(MULTIPOLYGON, 4283)                     
        );"""
))

conn.execute(text("""
    DROP TABLE IF EXISTS catchments_secondary;
    CREATE TABLE catchments_secondary (
         useid INTEGER,
         catchtype VARCHAR(100),
         usedesc VARCHAR(100),
         geom GEOMETRY(MULTIPOLYGON, 4283)                     
);"""))

conn.execute(text("""
    DROP TABLE IF EXISTS catchments_future;
    CREATE TABLE catchments_future (
         useid INTEGER,
         catchtype VARCHAR(100),
         usedesc VARCHAR(100),
         geom GEOMETRY(MULTIPOLYGON, 4283)                     
);"""
))

conn.commit()

In [26]:
income.to_sql("income", conn, if_exists='replace', index=False)
query(conn, "select * from income")
conn.commit()

In [27]:
stops.to_sql('stops', conn, if_exists='replace', index=False, dtype={'geom': Geometry('POINT', 4283)})
query(conn, "select * from stops")
conn.commit()

In [28]:
population.to_sql("population", conn, if_exists='replace', index=False)
query(conn, "select * from population")
conn.commit()

In [29]:
businesses.to_sql("businesses", conn, if_exists='replace', index=False)
query(conn, "select * from businesses")
conn.commit()

In [30]:
sa2_regions.to_sql('sa2_regions', conn, if_exists='replace', index=False, dtype={'geom': Geometry('MULTIPOLYGON', 4283)})
query(conn, "select * from sa2_regions")
conn.commit()

In [31]:
catchments_future.to_sql('catchments_future', conn, if_exists='replace', index=False, dtype={'geom': Geometry('MULTIPOLYGON', 4283)})
query(conn, "select * from catchments_future")
conn.commit()

In [32]:
catchments_primary.to_sql('catchments_primary', conn, if_exists='replace', index=False, dtype={'geom': Geometry('MULTIPOLYGON', 4283)})
query(conn, "select * from catchments_primary")
conn.commit()

In [33]:
catchments_secondary.to_sql('catchments_secondary', conn, if_exists='replace', index=False, dtype={'geom': Geometry('MULTIPOLYGON', 4283)})
query(conn, "select * from catchments_secondary")
conn.commit()

### Creating Indexes

In [ ]:
cluster = """

CLUSTER primar
"""

In [126]:
# Check the indexes  
sql = """
select * from pg_indexes
where schemaname = 'public'
"""
query(conn, sql)

,schemaname,tablename,indexname,tablespace,indexdef
0,public,hd_poi,idx_hd_poi_geom,None,CREATE INDEX idx_hd_poi_geom ON public.hd_poi ...
1,public,stops,idx_stops_geom,None,CREATE INDEX idx_stops_geom ON public.stops US...
2,public,sa2_regions,idx_sa2_regions_geom,None,CREATE INDEX idx_sa2_regions_geom ON public.sa...
3,public,spatial_ref_sys,spatial_ref_sys_pkey,None,CREATE UNIQUE INDEX spatial_ref_sys_pkey ON pu...
4,public,catchments_future,idx_catchments_future_geom,None,CREATE INDEX idx_catchments_future_geom ON pub...
5,public,catchments_primary,idx_catchments_primary_geom,None,CREATE INDEX idx_catchments_primary_geom ON pu...
6,public,catchments_secondary,idx_catchments_secondary_geom,None,CREATE INDEX idx_catchments_secondary_geom ON ...


# Task 2

### Extracting relevant information from NSW POI API

In [34]:
import requests 
from shapely.geometry import shape, box
import time

In [36]:
# Choosing SA4 regions
sa2_regions['sa4name'].unique()

# Chloe's region
Hills_district = sa2_regions[sa2_regions['sa4name'] == 'Sydney - Baulkham Hills and Hawkesbury']

In [ ]:
Hills_district
MULTIPOLYGON (((150.97069659849376 -33.75927015391956, 150.97071714850378 -33.759172563920785, 150.9708424885321 -33.758823673929925, 150.97101093850603 -33.75883897394559, 150.97123843847257 -33.758846783966675, 150.97147061843765 -33.758860323988245, 150.97164190841045 -33.75888124400422, 150.97160281843884 -33.75870917399941, 150.97154518847853 -33.758471763992404, 150.97202398841895 -33.75840583403621, 150.97218075839444 -33.758421924050815, 150.9722896384065 -33.75821356405942, 150.9726640583543 -33.75820473409397, 150.97266189835776 -33.7581809840936, 150.97333109825107 -33.758265674156064, 150.97355684819735 -33.758428034178074, 150.97357406819697 -33.758412514179554, 150.97363349818298 -33.75845402418535, 150.97392549822288 -33.757838864208026, 150.97406528819943 -33.75786524422113, 150.97418803835626 -33.75654952422323, 150.97442192828544 -33.75683233424686, 150.975911238028 -33.757171804386935, 150.97589499801794 -33.75726547438609, 150.97635577794478 -33.75732151442907, 150.9764339979915 -33.75688489443326, 150.97645111799798 -33.75681745443437, 150.97708193788912 -33.75695978449371, 150.97781237777252 -33.7570537445619, 150.977949907752 -33.757060564574665, 150.97812197772842 -33.75705318459054, 150.97829155770754 -33.75702808460603, 150.97845605768973 -33.75698569462094, 150.9788891676488 -33.756828194659896, 150.97890861764793 -33.75681398466159, 150.97892588764762 -33.75679777466308, 150.97893051764785 -33.756790954663465, 150.97900439763424 -33.75681414467046, 150.9792969775853 -33.756868444697886, 150.97960303754903 -33.756812984725805, 150.9796584775316 -33.75688498473143, 150.9800764775021 -33.7566575447685, 150.98016122749752 -33.75660093477593, 150.9802499574929 -33.756540364783724, 150.98110387747369 -33.755766414857256, 150.98141708746735 -33.755477464884194, 150.98120574753784 -33.7551728248625, 150.98116201755465 -33.7550930748579, 150.98152435752732 -33.75490948489012, 150.98158304752351 -33.754875294895314, 150.98165287752062 -33.7548217349014, 150.98171298752027 -33.75475972490651, 150.98206772752945 -33.75430874493616, 150.9827102075461 -33.75349199498985, 150.9835651775307 -33.75268837506328, 150.9835853675311 -33.75266335506497, 150.98360047753405 -33.75262528506609, 150.98360299753767 -33.75259503506612, 150.9835932875443 -33.75255554506494, 150.98356988755233 -33.75252026506253, 150.98365080754542 -33.752485385069775, 150.9839914975358 -33.75219116509922, 150.98352900764684 -33.75185130505405, 150.98374983764342 -33.75163945507298, 150.98399379764243 -33.751384395093766, 150.98432156766088 -33.750892525120626, 150.98400438775397 -33.75053162508874, 150.98409869776006 -33.75038428509643, 150.98427941776092 -33.75018339511175, 150.98447599773124 -33.750195825130014, 150.98485330776697 -33.74952065516018, 150.98495214777563 -33.749348885168104, 150.98480220780968 -33.74925312515355, 150.98449041788345 -33.74903192512318, 150.98383885804202 -33.74853656505942, 150.983553358112 -33.74831561503146, 150.9834083381483 -33.74819786501722, 150.98319536820358 -33.7480097249962, 150.98315240821577 -33.7479641349919, 150.98303516824632 -33.74785972498034, 150.98291368828012 -33.74773557496822, 150.98272867833438 -33.74752515494963, 150.98264446835844 -33.7474339449412, 150.9825848683767 -33.747360424935174, 150.98248686840816 -33.74722837492519, 150.98241283843086 -33.7471368349177, 150.98235764845074 -33.747046114911946, 150.98231016846674 -33.746976474907065, 150.98226234848372 -33.74689975490211, 150.9822067285048 -33.74680048489627, 150.98212284853773 -33.7466423848874, 150.98204371857113 -33.7464755948789, 150.98194753861645 -33.74623690486833, 150.98191395863506 -33.746132544864494, 150.98189468864922 -33.74604635486211, 150.9818679686709 -33.745911744858695, 150.98185511868073 -33.74585127485708, 150.9818533286912 -33.74577447485636, 150.9818455187051 -33.74567798485497, 150.98183799872066 -33.74556857485352, 150.98184157872694 -33.74551730485348, 150.9818519487398 -33.74540886485368, 150.98188176876357 -33.74519761485495, 150.98196782879927 -33.74483591486037, 150.98204367882934 -33.744527284865214, 150.9821165388587 -33.74422724486984, 150.98217248887818 -33.744020144873566, 150.98221774888964 -33.74388503487681, 150.9823047289086 -33.74364862488318, 150.9823822589237 -33.743450974888965, 150.98242323892987 -33.74336050489213, 150.9824641989364 -33.74326702489525, 150.98251737894313 -33.74315909489941, 150.982589578951 -33.74302197490513, 150.9826665489588 -33.742880114911266, 150.98271323896398 -33.74279111491495, 150.9827676889685 -33.742698554919336, 150.98283101897223 -33.74260191492451, 150.98288887897678 -33.74250530492918, 150.98301154898374 -33.74232100493923, 150.9831325489887 -33.742153634949254, 150.98325792899203 -33.74199363495971, 150.9833431589945 -33.74188326496682, 150.98350225899537 -33.74170542498029, 150.98360226899524 -33.74159891498879, 150.98387185899372 -33.741320125011775, 150.9839745289931 -33.74121493502052, 150.98409504899445 -33.7410750350307, 150.98415837899492 -33.74100325503605, 150.98422309899672 -33.74092007504146, 150.98427335899888 -33.74084988504561, 150.9846934389163 -33.741021145085675, 150.9848751689207 -33.74079251510087, 150.98544894879493 -33.74112435515629, 150.9853330587633 -33.7414873251481, 150.9853138287578 -33.74154966514676, 150.98557975871907 -33.74155599517142, 150.98571940869297 -33.74160289518466, 150.98684729845093 -33.74221585529331, 150.98726273836172 -33.74244214533333, 150.98736931833807 -33.74250604534363, 150.98784739823145 -33.742796295389894, 150.98838741811377 -33.743103745442006, 150.9886855880625 -33.74316951547006, 150.9894058379388 -33.74332837553781, 150.99005093779945 -33.743685675600005, 150.99026533775088 -33.74382167562081, 150.9903128977512 -33.74376798562482, 150.9903994377479 -33.74369988563235, 150.99042356774652 -33.74368432563447, 150.9905247077397 -33.74362718564343, 150.99063380773038 -33.743580195653195, 150.99074830771886 -33.74354409566353, 150.9908371877088 -33.74352439567161, 150.99110721767502 -33.74348874569636, 150.99225812753366 -33.74331765580165, 150.99269766747938 -33.74325457584187, 150.9928340974635 -33.743227585854314, 150.99295582745071 -33.74319309586532, 150.99307423743926 -33.743152255875984, 150.99318256742998 -33.74310568588569, 150.99324384740615 -33.74321971589215, 150.99317573738475 -33.7434544658875, 150.99309295737368 -33.74362681588106, 150.99216487735453 -33.7447690958032, 150.99209037735585 -33.74483902579681, 150.99294286717026 -33.74532241587908, 150.99368228700905 -33.74574324595045, 150.99459982680924 -33.74626385603902, 150.9944882068129 -33.74635604602935, 150.99423318681767 -33.74659447600742, 150.99404626681584 -33.74680903599164, 150.9938882968075 -33.74704167597864, 150.9937575167906 -33.74730988596843, 150.9937370767856 -33.74736962596695, 150.99369655677657 -33.74748125596398, 150.99436655663976 -33.74779245602817, 150.9947068465591 -33.748035106061366, 150.99537358640026 -33.74851597612645, 150.9962049862318 -33.74889263620602, 150.99665981613938 -33.749100446249585, 150.9968596460999 -33.74918369626867, 150.9972466260226 -33.74935036630565, 150.99767217593077 -33.74958508634669, 150.9977612359328 -33.749474316354146, 150.99782547591778 -33.74951830636039, 150.99784248590797 -33.749574306362376, 150.9979774658786 -33.74965030637539, 150.99819948588356 -33.749374306394, 150.9985174758148 -33.749551286424676, 150.9990614756973 -33.74985230647714, 150.9991774756721 -33.749917296488334, 150.99922948566098 -33.74994530649335, 150.9997504855484 -33.7502342965436, 150.99978848554932 -33.750186306546794, 151.00047150544773 -33.75021828661023, 151.00068349541633 -33.7502272966299, 151.00161147527817 -33.75027129671612, 151.001616475287 -33.75019928671607, 151.0018764652472 -33.75021993674027, 151.00187445524415 -33.75024529674026, 151.00185846524388 -33.75026429673892, 151.00188949523695 -33.750283296741934, 151.00201147521025 -33.750353306753716, 151.00209247520917 -33.75027428676067, 151.002149485204 -33.75025229676578, 151.00240647518103 -33.750149306788856, 151.00301048512677 -33.74990930684307, 151.0031994750855 -33.750017306861345, 151.00338646504437 -33.75012628687942, 151.00368350497945 -33.7502963069081, 151.0040494948993 -33.75050729694347, 151.00408244488477 -33.75058129694703, 151.00419388483692 -33.75082231695904, 151.0039484948314 -33.75112730693848, 151.00376270482752 -33.751356376922885, 151.00373752482682 -33.75138870692078, 151.00456147464826 -33.751849307000285, 151.0045784846486 -33.751828287001715, 151.0047234946172 -33.751909297015715, 151.0053944744714 -33.752287297080485, 151.005379474471 -33.752306297079244, 151.00578349438302 -33.75253530711824, 151.00532545437403 -33.75309529707976, 151.00548846433855 -33.75318729709551, 151.00565446430227 -33.753282297111554, 151.0063064641605 -33.7536502971745, 151.00679648405415 -33.7539253072218, 151.00761346387648 -33.75438630730068, 151.00794148380507 -33.75457230733235, 151.00810547376946 -33.754664307348186, 151.00892348359204 -33.75512228742714, 151.00925847352633 -33.75525730745911, 151.00907346353694 -33.75537629744282, 151.00896348354104 -33.755463297433245, 151.00885748354384 -33.755556297424086, 151.0087644835494 -33.75561429741588, 151.0087444835501 -33.75563029741414, 151.0086994835483 -33.75569229741043, 151.00867546354124 -33.755771307408736, 151.00866546353103 -33.75585929740844, 151.00865746350843 -33.75603829740894, 151.00868747349998 -33.756069307411956, 151.00871247349153 -33.75610630741453, 151.00873247348338 -33.75614629741665, 151.00874547347607 -33.756187297418144, 151.00875047347003 -33.75622729741889, 151.0087494834678 -33.75624529741891, 151.00874448346374 -33.7562812974187, 151.00873448346053 -33.756316297418024, 151.0087124734594 -33.75634830741622, 151.00868046346 -33.756378287413476, 151.0086434834614 -33.75640729741024, 151.0086104534623 -33.75643628740739, 151.0086054934619 -33.756444297406986, 151.00858746346117 -33.756469297405495, 151.00857647345794 -33.75650529740473, 151.0085714734535 -33.75654428740456, 151.00857147345084 -33.75656428740468, 151.00857848342875 -33.75672329740644, 151.0085834834205 -33.75678029740732, 151.00860447339466 -33.756952287410456, 151.00860647338786 -33.75700129741099, 151.00860647337458 -33.7571012874117, 151.0086124633675 -33.75714829741257, 151.00862848335936 -33.75719228741436, 151.00864246335433 -33.757215307415834, 151.00866046334897 -33.75723630741764, 151.0086994833377 -33.757279297421555, 151.00879248331026 -33.75738629743092, 151.00882847329942 -33.75742929743455, 151.00886248328888 -33.75747228743801, 151.00889249327844 -33.7575182974411, 151.00892048326688 -33.757575297444106, 151.00894246325578 -33.757635297446555, 151.00896049324527 -33.75769530744865, 151.00900447321575 -33.75787029745395, 151.00902349319927 -33.757974307456436, 151.00903546319077 -33.75802528745791, 151.0090504631819 -33.75807629745965, 151.00907046317266 -33.75812430746184, 151.00909149316317 -33.75817328746413, 151.00911547315334 -33.7582212974667, 151.00914346314326 -33.758267287469614, 151.009174493133 -33.758311297472794, 151.00923946311275 -33.75839428747938, 151.00927647310215 -33.758434287483105, 151.00931746309175 -33.75846828748713, 151.00936348308187 -33.75849330749157, 151.00940046307517 -33.75850429749507, 151.00944148306837 -33.75851130749892, 151.00956945304893 -33.75852029751084, 151.00962749304026 -33.75852329751623, 151.009742463024 -33.75852229752687, 151.00979948301563 -33.75852428753216, 151.00985547300684 -33.75853030753738, 151.0099124829971 -33.758542287542745, 151.00996847298697 -33.75855829754805, 151.01002446297662 -33.75857629755336, 151.0100764729663 -33.75859828755833, 151.01012349295627 -33.75862330756286, 151.01018046294254 -33.75866529756843, 151.01022846292932 -33.75871330757321, 151.01027248291604 -33.758766287577664, 151.01031248290286 -33.75882228758175, 151.01034847289017 -33.75887930758548, 151.01037147288176 -33.7589183075879, 151.01039246287345 -33.75895829759011, 151.01041147286526 -33.75899929759216, 151.0104284828573 -33.75904129759404, 151.01044348284955 -33.75908330759572, 151.01041147285412 -33.75908330759277, 151.01033048285203 -33.75918629758598, 151.01018846284705 -33.75937628757416, 151.01012246283895 -33.75950830756898, 151.01004549282587 -33.759689287563134, 151.00997048281124 -33.75988030755751, 151.0099214627703 -33.76024129755551, 151.00985846274426 -33.76050528755154, 151.00981346272212 -33.760720297548886, 151.00980846269792 -33.76090829754974, 151.00982747264845 -33.76126029755396, 151.0098744825832 -33.761701297561416, 151.00988346252552 -33.76212629756524, 151.00978046248022 -33.76257828755888, 151.00963647243995 -33.76303629754875, 151.00933147237487 -33.763854287526264, 151.00928348236565 -33.76397529752266, 151.00926048235246 -33.76409930752141, 151.00917646231707 -33.76445629751613, 151.00910846230423 -33.764626297511036, 151.00900447229284 -33.76482329750279, 151.0089544622866 -33.76492428749887, 151.00887749227462 -33.76509730749296, 151.0087644822601 -33.76532829748411, 151.00862348225644 -33.76550729747231, 151.0085104822626 -33.76558229746238, 151.00813548231332 -33.765603297427816, 151.00737748240604 -33.76571928735843, 151.00659346258584 -33.765207307282246, 151.00599946257432 -33.76593229723233, 151.0051769325583 -33.76693710716326, 151.00399160270808 -33.76708244705454, 151.00325707279822 -33.76719277698731, 151.00273698286205 -33.767270906939714, 151.0022682129218 -33.767324886896695, 151.0019846129579 -33.76735755687069, 151.00176101298658 -33.767381866850165, 151.00153423301518 -33.767410306829355, 151.00127249304873 -33.76743875680533, 151.0009726530864 -33.767477176777845, 151.00062487313107 -33.76751443674591, 151.00045102315292 -33.76753665672998, 151.00034769316574 -33.76755116672053, 151.000121683194 -33.767581166699806, 150.99990163322113 -33.767613176679674, 150.99968022324816 -33.76764736665942, 150.9986137233725 -33.767857186562175, 150.99877013332565 -33.768042186577944, 150.99823212338353 -33.76818417652915, 150.9981277333946 -33.7682132065197, 150.99762173344922 -33.76834549647379, 150.9973627634772 -33.7684132064503, 150.99726976348717 -33.76843819644186, 150.99577577364857 -33.768828236306355, 150.995060813726 -33.76901325624148, 150.99416280382442 -33.769237266159955, 150.99375784387027 -33.76932725612312, 150.99214786404542 -33.76973828597702, 150.9908198841864 -33.77010428585672, 150.98959490431685 -33.77043832574572, 150.98854096443142 -33.77070833565013, 150.98593099471555 -33.77137435541336, 150.98550300476202 -33.771484375374534, 150.98530100478408 -33.771535375356216, 150.98396156493035 -33.77187350523469, 150.98224435511872 -33.77230134507886, 150.98222336512117 -33.77230527507693, 150.98211080513445 -33.77232628506669, 150.98060977533078 -33.77246130492882, 150.97916382551992 -33.772591334796005, 150.9773947457475 -33.77277898463373, 150.9754056760034 -33.772989884451285, 150.97536392606878 -33.772541424444285, 150.9749055561314 -33.77256258440204, 150.97465082616648 -33.77257248437856, 150.97433436621054 -33.77258075434935, 150.9740802562468 -33.77258070432587, 150.97370017630269 -33.77256837429064, 150.97344643634094 -33.77255296426707, 150.97319355637967 -33.77253302424355, 150.972941526419 -33.77250762422006, 150.97256528647893 -33.77246087418495, 150.97225266653012 -33.77241118415569, 150.97194200658163 -33.77235696412661, 150.97169443662324 -33.77230974410339, 150.9715098166549 -33.77226954408604, 150.97120671670865 -33.77219068405747, 150.97090643676313 -33.77210269402908, 150.9706101268182 -33.77200644400101, 150.97031571687356 -33.77190563397309, 150.97002535692923 -33.77179835394549, 150.96956815701913 -33.77161239390193, 150.96934404706442 -33.77151206388051, 150.96906880712123 -33.77137990385413, 150.9687975371784 -33.771240433828076, 150.9685856572243 -33.77112254380767, 150.96837993727013 -33.770998133787785, 150.96831000728636 -33.770951143780984, 150.96808213733914 -33.770798073758854, 150.96784382739642 -33.77062230373559, 150.96779035740914 -33.77058412373037, 150.96728617753473 -33.770179453680925, 150.96688599763968 -33.76981831364141, 150.96651438774333 -33.76943638360437, 150.96615318785138 -33.769010213568, 150.96616252785483 -33.76897377356861, 150.96598710791787 -33.76868691355039, 150.96593659793518 -33.76861089354518, 150.96588527795305 -33.76853132353987, 150.9658353079708 -33.768451183534694, 150.96578654798844 -33.76837050352962, 150.965739098006 -33.76828929352467, 150.9656863480259 -33.768195683519124, 150.96556217807498 -33.767959413506, 150.96544333812534 -33.76770715349324, 150.96518553824706 -33.7670659034649, 150.9650900882885 -33.76685605345462, 150.96495236834036 -33.76661306344018, 150.96480111839 -33.76640153342472, 150.9646767284271 -33.76625547341221, 150.96454086846484 -33.76611672339866, 150.9643941785031 -33.765985933384194, 150.96434811851447 -33.76594976337968, 150.96423726854172 -33.76586368336884, 150.96407089858042 -33.76575050335266, 150.96389573861913 -33.76564691333574, 150.96369231866174 -33.765544483316226, 150.9637561586928 -33.765241393320004, 150.96346291880025 -33.76474628328941, 150.96324051888075 -33.764378073266286, 150.96396292878796 -33.7643003733325, 150.9640855387697 -33.76430603334387, 150.96408885877094 -33.76429332334409, 150.96411403877607 -33.764227193345945, 150.9641233887806 -33.7641829933465, 150.9641626187839 -33.764116113349665, 150.96423715878723 -33.76401055335581, 150.9643196087888 -33.763909653362724, 150.96437793878908 -33.76384468336766, 150.96447151878823 -33.763750533375635, 150.9645359787869 -33.76369106338119, 150.96470767878105 -33.76355029339606, 150.96481713877583 -33.76347190340562, 150.96493160876926 -33.76339828341568, 150.96505004876144 -33.76332970342615, 150.9651922187509 -33.763256343438776, 150.96543811873025 -33.76314718346074, 150.96559038871615 -33.763089683474405, 150.96569372870601 -33.763054933483716, 150.96585501868765 -33.76301965349837, 150.96580172871555 -33.762866733492366, 150.96577829873 -33.76278304348963, 150.96576212874345 -33.76269872348753, 150.96575347875597 -33.76261358348614, 150.96575277876735 -33.76252817348548, 150.96580424883177 -33.76198663348642, 150.9659467790072 -33.760509093489226, 150.96595867901317 -33.76045106348991, 150.9659781290179 -33.760394453491315, 150.9660159090221 -33.760322173494295, 150.96606594902377 -33.76025575349845, 150.9663636490181 -33.759977943524014, 150.96638838901222 -33.759995503526426, 150.9664982190045 -33.759935613536165, 150.9667028090061 -33.759703233553445, 150.96796741877176 -33.76011051367319, 150.96857875867494 -33.760182753730206, 150.96867317871119 -33.7598075737363, 150.9690046386978 -33.75955168376514, 150.969447088619 -33.75967012380688, 150.96957169859502 -33.75971687381871, 150.96958604859395 -33.75970982381999, 150.97030640853723 -33.75936231388414, 150.97045507852192 -33.75931758389757, 150.97053102851353 -33.75929914390446, 150.97064589850012 -33.759276733914916, 150.97069659849376 -33.75927015391956)))

,sa2code,sa2name,sa3code,sa3name,sa4code,sa4name,areasqkm,geom
299,115011290,Baulkham Hills - East,11501,Baulkham Hills,115,Sydney - Baulkham Hills and Hawkesbury,8.3081,MULTIPOLYGON (((150.97069659849376 -33.7592701...
300,115011291,Baulkham Hills (West) - Bella Vista,11501,Baulkham Hills,115,Sydney - Baulkham Hills and Hawkesbury,10.8721,MULTIPOLYGON (((150.94715049428663 -33.7408813...
301,115011294,Glenhaven,11501,Baulkham Hills,115,Sydney - Baulkham Hills and Hawkesbury,7.2309,MULTIPOLYGON (((150.98461766397278 -33.7029313...
302,115011296,West Pennant Hills,11501,Baulkham Hills,115,Sydney - Baulkham Hills and Hawkesbury,7.8291,MULTIPOLYGON (((151.0221504638731 -33.73880029...
303,115011553,Castle Hill - Central,11501,Baulkham Hills,115,Sydney - Baulkham Hills and Hawkesbury,4.1512,MULTIPOLYGON (((150.97343625140343 -33.7343577...
304,115011554,Castle Hill - East,11501,Baulkham Hills,115,Sydney - Baulkham Hills and Hawkesbury,2.0720,MULTIPOLYGON (((151.02112670725464 -33.7144151...
305,115011555,Castle Hill - North,11501,Baulkham Hills,115,Sydney - Baulkham Hills and Hawkesbury,6.8149,MULTIPOLYGON (((150.98369647386426 -33.7047412...
306,115011556,Castle Hill - South,11501,Baulkham Hills,115,Sydney - Baulkham Hills and Hawkesbury,4.1277,MULTIPOLYGON (((151.00308464710827 -33.7348903...
307,115011557,Castle Hill - West,11501,Baulkham Hills,115,Sydney - Baulkham Hills and Hawkesbury,1.6363,MULTIPOLYGON (((150.96740547459288 -33.7167573...
308,115011558,Cherrybrook,11501,Baulkham Hills,115,Sydney - Baulkham Hills and Hawkesbury,8.2514,MULTIPOLYGON (((151.0339003548186 -33.71907059...


In [37]:
# define the bounding box for each sa2 region within the Hills District
sql = """
select sa2name,
ST_XMIN(ST_EXTENT(geom)) as xmin, ST_YMIN(ST_EXTENT(geom)) as ymin,
ST_XMAX(ST_EXTENT(geom)) as xmax, ST_YMAX(ST_EXTENT(geom)) as ymax
FROM sa2_regions
WHERE sa4name = 'Sydney - Baulkham Hills and Hawkesbury'
GROUP BY sa2name"""

HD_bboxes = query(conn,sql)

In [ ]:
# same query for the other sa4 regions

In [38]:
# ls of dictionaries containing sa2 region and their corresponding bounding box
HD_bboxes = HD_bboxes.apply(lambda row: {"sa2name": row['sa2name'],
                                         "bbox": (row["xmin"], row["ymin"], row["xmax"], row["ymax"])},
                                          axis=1).values.tolist()

# do the same for other regions 

In [39]:
# Adapted code from wk 7 NB
# use bounding box as param instead of a midpoint
# outfields include topoid, poiname, poitype and poigroup only
def POI_in_bbox(bbox, filters={}):
    baseURL = 'https://maps.six.nsw.gov.au/arcgis/rest/services/public/NSW_POI/MapServer/0/query'

    xmin,ymin, xmax, ymax = bbox
    params = {
           'geometry': f'"xmin":{xmin},"ymin":{ymin},"xmax":{xmax},"ymax:{ymax}"',
            'outFields': 'topoid,poiname,poitype,poigroup',
            'returnGeometry': 'true',
            'f': 'json'
        }
    response = requests.get(baseURL, params)
    return json.loads(response.text)['features']


In [40]:
HDpoi_results = {}
for entry in HD_bboxes:
    sa2name = entry["sa2name"]
    bbox = entry["bbox"]
    
    time.sleep(5)  # waits 5-second before sending a request
    HDpoi_results[sa2name] = POI_in_bbox(bbox)

#encapsulate function and fetch POI for the other regions

In [41]:
# Store the outputs in a form that can be converted into a pandas dataframe
rows = []
for sa2name, pois in HDpoi_results.items():
    for poi in pois:
        rows.append({
            'sa2name': sa2name,
            'topoid': poi['attributes']['topoid'],
            'poiname': poi['attributes']['poiname'],
            'poitype': poi['attributes']['poitype'],
            'poigroup': poi['attributes']['poigroup'],
            'lng': poi['geometry']['x'],
            'lat': poi['geometry']['y']
        })

hd_poi = pd.DataFrame(rows)

### Cleaning dataset

In [42]:
# Creating the geometry column 
hd_poi['geom'] = gpd.points_from_xy(hd_poi.lng, hd_poi.lat) 

# Dropping the old lat and lng columns 
hd_poi.drop(columns=['lat', 'lng'],axis=1,inplace=True)


In [43]:
# dealing with null values
hd_poi['poiname'] = hd_poi['poiname'].fillna('UNNAMED')

In [44]:
srid = 4283
# NB week 8
hd_poi['geom'] = hd_poi['geom'].apply(lambda x: WKTElement(x.wkt, srid=srid))

### Ingesting into SQL

In [45]:
conn.execute(text("""
    DROP TABLE IF EXISTS hd_poi;
    CREATE TABLE hd_poi (
         topoid INTEGER UNIQUE,
         sa2name VARCHAR(100),
         poiname VARCHAR(100),
         poitype VARCHAR(100),
         poigroup VARCHAR(100),
         geom GEOMETRY(POINT, 4283)                     
);"""
))
conn.commit()

In [46]:

hd_poi.to_sql('hd_poi', conn, if_exists='replace', index=False, dtype={'geom': Geometry('POINT', 4283)})
query(conn, "select * from hd_poi")
conn.commit()

# Task 3

### Queries

In [140]:
conn.rollback()
# Create a view for bounding box of each sa2 region within the Hills District
# encapsulation enable readable queries
view = """ 
drop view if exists HillsDistrict;
create view HillsDistrict as 
select sa2code, 
       sa2name, 
       areasqkm,
       geom,
       ST_MakeEnvelope(
            ST_XMin(ST_EXTENT(geom)), ST_YMin(ST_EXTENT(geom)), 
            ST_XMax(ST_EXTENT(geom)), ST_YMax(ST_EXTENT(geom)), 4283
        ) AS bbox
from sa2_regions
where sa4name = 'Sydney - Baulkham Hills and Hawkesbury'
group by sa2code, sa2name, areasqkm, geom
"""
query(conn,view)
conn.commit()

Error encountered: 
This result object does not return rows. It has been closed automatically.


In [ ]:
# Functions definition
"""
Sigmoid function = 1/(1+e^(sum of Z_scores))
where:
Z = \frac{x - \mu}{\sigma}
Z	=	standard score
x	=	observed value
\mu	=	mean of the sample
\sigma	=	standard deviation of the sample
"""

##### a. Calculating the z-score for businesses by sa2 region

In [ ]:
businesses.columns
businesses['industry_name'].unique()

# According to the Hills Shire Industry Sector of Employment, the top 3 industries are:
# Healthcare and Social Assistance
# Professional, Scientific and Technical Services
# Education and Training 

In [ ]:
# For the business metric, we define being well-resourced as 
# the working population's accessibility to economic opportunities  
## Therefore add a column called working_people to the population table
## working population is anyone aged 15-64 

conn.rollback()
sql = """
alter table population 
add working_population INT;

update population 
set working_population = age_15_to_19 + age_20_to_24 + age_25_to_29 + age_30_to_34 +
                         age_35_to_39+ age_40_to_44 + age_45_to_49 + 
                         age_50_to_54 + age_55_to_59 + age_60_to_64
"""
query(conn,sql)
conn.commit()

In [190]:
conn.rollback()
sql = """
with filtered_businesses as(
select hd.sa2code,
       hd.sa2name,
       sum(b.total_businesses) as "total_industry_businesses"
from HillsDistrict hd
join businesses b on hd.sa2code::BIGINT = b.sa2_code
where b.industry_name in ( 'Healthcare and Social Assistance',
                          'Professional, Scientific and Technical Services',
                           'Education and Training'
)
group by  hd.sa2code, hd.sa2name
),
businesses_percap as(
select b.sa2code, 
       b.sa2name,
       (b.total_industry_businesses / p.working_population)*1000 as "businesses_per_cap"
from filtered_businesses b 
join population p on b.sa2code::BIGINT = p.sa2code
group by b.sa2code, b.sa2name, b.total_industry_businesses, p.working_population
)
select bp.sa2code, 
       bp.sa2name,
       total_industry_businesses,
       (bp.businesses_per_cap - avg(bp.businesses_per_cap) OVER())/stddev_samp(businesses_per_cap) OVER() as "businesses_zscore"
from businesses_percap bp 
join filtered_businesses f on f.sa2code= bp.sa2code
group by bp.sa2code, bp.sa2name, bp.businesses_per_cap, total_industry_businesses
order by (bp.businesses_per_cap - avg(bp.businesses_per_cap) OVER())/stddev_samp(businesses_per_cap) OVER() desc
"""

query(conn,sql)

,sa2code,sa2name,total_industry_businesses,businesses_zscore
0,115011291,Baulkham Hills (West) - Bella Vista,1080.0,2.633528
1,115011553,Castle Hill - Central,324.0,2.008733
2,115011296,West Pennant Hills,389.0,0.803507
3,115021298,Galston - Laughtondale,158.0,0.757408
4,115011556,Castle Hill - South,277.0,0.319612
5,115011294,Glenhaven,146.0,0.286410
6,115011557,Castle Hill - West,141.0,0.222838
7,115011554,Castle Hill - East,113.0,0.193733
8,115021297,Dural - Kenthurst - Wisemans Ferry,543.0,0.120815
9,115011558,Cherrybrook,460.0,0.083341


##### b. Calculating the z-score for schools by sa2 region

In [116]:
# Add column called young people to the population table
# # young people is anyone aged 0-19
conn.rollback()
sql = """
alter table population 
add young_people INT;

update population 
set young_people = age_0_to_4 + age_5_to_9 + age_10_to_14 + age_15_to_19
"""
query(conn,sql)

conn.commit()

Error encountered: 
This result object does not return rows. It has been closed automatically.


In [ ]:
conn.rollback()
# Schools z score for each sa2 region in the Hills District 
# Decided to go with ST_Intersects rather than ST_Within 
sql ="""
with all_catchments as(
select * from catchments_primary 
union all
select * from catchments_secondary
union all 
select * from catchments_future
),
total_catch as(
select hd.sa2code, 
       hd.sa2name, 
       count(ac.useid) as "total_catchments"
from HillsDistrict hd
join all_catchments ac on ST_Intersects(ac.geom, hd.geom)
group by hd.sa2code, hd.sa2name
),
all_catch_percap as(
select tc.sa2code, 
       tc.sa2name,
       p.young_people,
       tc.total_catchments,
       (tc.total_catchments::numeric/p.young_people)*1000.0 as "catchments_per_cap"
from total_catch tc
join population p on tc.sa2code::BIGINT = p.sa2code::BIGINT
)
select c.sa2code, 
       c.sa2name, 
       c.catchments_per_cap,
       (c.catchments_per_cap - avg(c.catchments_per_cap) OVER())/ stddev_samp(c.catchments_per_cap) OVER() as "schools_zscore"
from all_catch_percap c 
order by (c.catchments_per_cap - avg(c.catchments_per_cap) OVER())/ stddev_samp(c.catchments_per_cap) OVER() desc
"""
query(conn,sql)

,sa2code,sa2name,catchments_per_cap,schools_zscore
0,115031299,Bilpin - Colo - St Albans,23.178808,3.640519
1,115021298,Galston - Laughtondale,12.592593,1.495995
2,115011554,Castle Hill - East,7.042254,0.371624
3,115011294,Glenhaven,6.361323,0.233683
4,115021297,Dural - Kenthurst - Wisemans Ferry,5.977797,0.155990
5,115041301,Pitt Town - McGraths Hill,5.866008,0.133344
6,115011553,Castle Hill - Central,5.649718,0.089528
7,115011296,West Pennant Hills,4.241436,-0.195757
8,115011556,Castle Hill - South,4.059041,-0.232706
9,115031300,Kurrajong Heights - Ebenezer,3.705575,-0.304310


##### c. Calculating the z score for stops by sa2 region

In [ ]:
Hills_district.sort_values("areasqkm",ascending=False)
# Note that the sa2 regions are very variable in land area.
# The top 3 sa2 regions were much bigger than the rest.


,sa2code,sa2name,sa3code,sa3name,sa4code,sa4name,areasqkm,geom
313,115031299,Bilpin - Colo - St Albans,11503,Hawkesbury,115,Sydney - Baulkham Hills and Hawkesbury,2160.1601,MULTIPOLYGON (((150.9752845498258 -33.36518230...
311,115021297,Dural - Kenthurst - Wisemans Ferry,11502,Dural - Wisemans Ferry,115,Sydney - Baulkham Hills and Hawkesbury,377.5396,MULTIPOLYGON (((150.9344355080788 -33.65019828...
314,115031300,Kurrajong Heights - Ebenezer,11503,Hawkesbury,115,Sydney - Baulkham Hills and Hawkesbury,331.8500,MULTIPOLYGON (((150.61982399861674 -33.5287535...
312,115021298,Galston - Laughtondale,11502,Dural - Wisemans Ferry,115,Sydney - Baulkham Hills and Hawkesbury,191.6990,MULTIPOLYGON (((151.1520473634862 -33.52765932...
315,115041301,Pitt Town - McGraths Hill,11504,Rouse Hill - McGraths Hill,115,Sydney - Baulkham Hills and Hawkesbury,80.5030,MULTIPOLYGON (((150.92995550346822 -33.6141572...
316,115041623,Box Hill - Nelson,11504,Rouse Hill - McGraths Hill,115,Sydney - Baulkham Hills and Hawkesbury,21.0706,MULTIPOLYGON (((150.88202821625538 -33.6446571...
300,115011291,Baulkham Hills (West) - Bella Vista,11501,Baulkham Hills,115,Sydney - Baulkham Hills and Hawkesbury,10.8721,MULTIPOLYGON (((150.94715049428663 -33.7408813...
318,115041625,Rouse Hill - Beaumont Hills,11504,Rouse Hill - McGraths Hill,115,Sydney - Baulkham Hills and Hawkesbury,8.6462,MULTIPOLYGON (((150.9140094870545 -33.67996928...
299,115011290,Baulkham Hills - East,11501,Baulkham Hills,115,Sydney - Baulkham Hills and Hawkesbury,8.3081,MULTIPOLYGON (((150.97069659849376 -33.7592701...
308,115011558,Cherrybrook,11501,Baulkham Hills,115,Sydney - Baulkham Hills and Hawkesbury,8.2514,MULTIPOLYGON (((151.0339003548186 -33.71907059...


In [ ]:
# The z-score for stops for each sa2 region in the Hills District 
## Extended the stops metric so that it is density-based (total number of stops/areasqkm).
## A larger region naturally have more space for stops but this does not mean it is well-resourced.
## A large region with a high stops count could still be low-resourced if the resources are spread out. 
sql ="""
with HD_stops as(
select hd.sa2code, hd.sa2name, 
       count(stop_id)/areasqkm as "stops_per_sqkm"
from HillsDistrict hd 
join stops s on ST_Within(s.geom, hd.bbox) 
group by hd.sa2name, hd.sa2code, hd.areasqkm
)
select hds.*, 
       (hds.stops_per_sqkm - avg(stops_per_sqkm) OVER())/stddev_samp(hds.stops_per_sqkm) OVER() as "stops_zscore"
from HD_stops hds 
order by (hds.stops_per_sqkm - avg(stops_per_sqkm) OVER())/stddev_samp(hds.stops_per_sqkm) OVER() desc
"""
query(conn,sql)

,sa2code,sa2name,stops_per_sqkm,stops_zscore
0,115011554,Castle Hill - East,70.945946,1.640785
1,115011290,Baulkham Hills - East,59.941503,1.128833
2,115011553,Castle Hill - Central,52.996724,0.805747
3,115011291,Baulkham Hills (West) - Bella Vista,51.048096,0.715092
4,115011622,Kellyville - West,50.255281,0.678209
5,115011557,Castle Hill - West,47.668520,0.557867
6,115041625,Rouse Hill - Beaumont Hills,46.957045,0.524767
7,115011558,Cherrybrook,46.537557,0.505252
8,115011296,West Pennant Hills,46.365483,0.497246
9,115011621,Kellyville - East,45.141612,0.440309


##### d. Calculating the z score for POI groups by sa2 region

In [ ]:
# According to NSW Topographic Data Dictionary, each of the POI group represents:
# 1 - Community, 2 - Education, 3 - Recreation, 4- Tranportation, 5 - Utility
# 6 - Hydrography, 7 - Landform, 8 - Place, 9 - Industry Facility

recreation = hd_poi[hd_poi['poigroup'] == 3]
recreation['poitype'].unique()

community = hd_poi[hd_poi['poigroup'] ==1]
community['poitype'].unique()


# Combined, POI group 1 and 3 include a large range of services that cater to most age groups. 
# Accessibility to these services

array(['Park', 'Sports Field', 'Sports Court', 'Club',
       'Swimming Pool Facility', 'Golf Course', 'Sports Centre',
       'Historic Site', 'Picnic Area', 'Lookout', 'Monument',
       'Tourist Attraction', 'Camping Ground', 'Showground',
       'Target Range', 'Racecourse'], dtype=object)

In [ ]:
conn.rollback()
# The z score for POI groups for each sa2 region in the Hills District
## Similarly extended the POI metric so that it is density based (total number of POI groups/areasqkm)
sql = """
with g1poi_count as (
select hd.sa2code, 
       hd.sa2name,
       p.poigroup,
       count(p.topoid)/hd.areasqkm as "g1poi_per_sqkm"
from HillsDistrict hd 
join hd_poi p on ST_Within(p.geom, hd.bbox) 
where poigroup =1 
group by hd.sa2name, hd.sa2code, p.poigroup, hd.areasqkm
),
g3poi_count as (
select hd.sa2code, 
       hd.sa2name,
       p.poigroup,
       count(p.topoid)/hd.areasqkm as "g3poi_per_sqkm"
from HillsDistrict hd 
join hd_poi p on ST_Within(p.geom, hd.bbox)
where poigroup = 3
group by hd.sa2name, hd.sa2code, p.poigroup, hd.areasqkm
)
select g.sa2code, 
       g.sa2name,
       g.g1poi_per_sqkm + g3.g3poi_per_sqkm as "poi_count",
       ((g.g1poi_per_sqkm + g3.g3poi_per_sqkm) - avg(g.g1poi_per_sqkm + g3.g3poi_per_sqkm) OVER())/ stddev(g.g1poi_per_sqkm + g3.g3poi_per_sqkm) OVER() as "POI_zscore"
from g1poi_count g
join g3poi_count g3 on g.sa2code = g3.sa2code
order by ((g.g1poi_per_sqkm + g3.g3poi_per_sqkm) - avg(g.g1poi_per_sqkm + g3.g3poi_per_sqkm) OVER())/ stddev(g.g1poi_per_sqkm + g3.g3poi_per_sqkm) OVER() desc
"""
query(conn,sql)

,sa2code,sa2name,poi_count,POI_zscore
0,115011621,Kellyville - East,41.481481,1.776421
1,115011294,Glenhaven,30.286686,0.881733
2,115011553,Castle Hill - Central,30.111775,0.867754
3,115011556,Castle Hill - South,30.040943,0.862093
4,115011555,Castle Hill - North,29.787671,0.841851
5,115041625,Rouse Hill - Beaumont Hills,29.145752,0.790549
6,115011622,Kellyville - West,28.831715,0.765451
7,115011554,Castle Hill - East,27.027027,0.621220
8,115011557,Castle Hill - West,26.889935,0.610264
9,115011290,Baulkham Hills - East,25.757995,0.519799


In [216]:
conn.close()
db.dispose()

# Appendix

### Resources:
The following were referenced to decide which POI groups and industries were included in calculating the respective metrics. 

1. POI API data dictionary: https://www.spatial.nsw.gov.au/__data/assets/pdf_file/0008/232559/NSW_Topographic_Data_Dictionary_v10.pdf

2. The Hills Shire Indsutry of Employment: https://profile.id.com.au/the-hills/industries 